# 📈 Régression et modèles d’ensemble – Prédiction du prix des maisons

La régression linéaire est l’un des algorithmes les plus simples en apprentissage automatique. Elle repose sur l’hypothèse qu’il existe une relation linéaire entre les variables explicatives et la variable à prédire.

Dans ce cahier, nous allons comparer deux types d’algorithmes de régression :

> 1. Régression linéaire: un modèle linéaire qui cherche à ajuster un hyperplan aux données.
> 2. Régresseur de forêt aléatoire (Random Forest Regressor): un modèle non linéaire basé sur un ensemble d’arbres de décision, capable de capturer des relations complexes entre les variables.

# 💾 Données

> Nous allons utiliser l'ensemble de données `USA_Housing`. Étant donné que le prix des maisons est une variable continue, il s'agit d'un problème de régression. Les données contiennent les colonnes suivantes :
> * '`Avg. Area Income`': Le revenu moyen des habitants de la ville où se trouve la maison.
> * '`Avg. Area House Age`': Âge moyen des maisons dans la même ville.
> * '`Avg. Area Number of Rooms`': Nombre moyen de pièces pour les maisons dans la même ville.
> * '`Avg. Area Number of Bedrooms`': Nombre moyen de chambres pour les maisons dans la même ville.
> * '`Area Population`': La population de la ville où se trouve la maison.
> * '`Price`': Prix auquel la maison a été vendue.
> * '`Address`': Adresse de la maison.


# Installation des Librairies

In [ ]:
%pip install -q hvplot
%pip install -q matplotlib
%pip install -q seaborn
%pip install -q scikit-learn

# 📤 Import des Libraries

In [2]:
import pandas as pd # Analyse et manipulation de données (utilisation des Panda DataFrame) https://pandas.pydata.org/docs/user_guide/index.html and/or https://sparkbyexamples.com/python-pandas-tutorial-for-beginners/ 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # Visualization de données
import hvplot.pandas #Plots interactifs https://hvplot.holoviz.org/

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 💾 Consultation des données

Allons récupérer les données à partir du fichier CSV `'USA_housing.csv'`

In [3]:
USAhousing = pd.read_csv('USA_Housing.csv')
type(USAhousing)

pandas.core.frame.DataFrame

**Instruction:** Affichez les 5 premières lignes
<details>
<summary>Aide</summary>
Utilisez la méthode <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html">head</a> des pandas.DataFrame.
</details>

In [ ]:
## Code ici


Vérifions les informations suivantes: 
- Nombre de lignes et de colonnes
- Type des colonnes
- Nombre de valeurs manquantes par colonne

In [ ]:
USAhousing.info()

**Instruction:** Calculer les statistiques suivantes pour chaque colonne:
- moyenne, écart type
- valeur minimale, maximale
- quartiles

<details>
<summary>Aide</summary>
Utilisez la méthode <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html">describe</a> des pandas.DataFrame.
</details>

In [ ]:
# Code ici


# 📊 Analyse exploratoire des données (AED)

Créons quelques graphiques simples pour examiner les données !

**Instruction:** Visualisez la distribution des prix des maisons à l'aide d'un histogramme
<details>
<summary>Aide</summary>
Vous pouvez utiliser la méthode  <a href="https://hvplot.holoviz.org/reference/tabular/hist.html">hvplot.hist</a> de la librairie hvplot.pandas
</details>

In [ ]:
# Code ici


Et maintenant visualisons la façon dont les prix des maisons évoluent en fonction de l'age moyen des maisons dans la même ville

In [ ]:
USAhousing.hvplot.scatter(x='Avg. Area House Age', y='Price')

**Instruction:** Visualisez la façon dont les prix évoluent en fonction du revenu moyen des habitants de la ville où se trouve la maison

In [ ]:
# Code ici


On peut en fait tracer en une seule figure les histogrammes de toutes les variables et visualiser l'évolution de chaque variable en fonction de toutes les autres!  

In [ ]:
sns.pairplot(USAhousing)

Calculons la matrice de corrélation entre toutes les colonnes du dataframe `'USAhousing'` (à l'exception de la colonne `'Address'`)

In [ ]:
correlation_matrix = USAhousing.iloc[:,USAhousing.columns!='Address'].corr()

**Instruction**: Visualisez la matrice de corrélation à l'aide d'une heatmap


<details>
<summary>Aide</summary>
Voir <a href="https://seaborn.pydata.org/generated/seaborn.heatmap.html"> sns.heatmap</a>
</details>

In [ ]:
# Code ici


# 📈 Entraînement du modèle de régression linéaire

> Commençons maintenant à entraîner notre modèle de régression ! 

> Nous devrons d'abord diviser nos données en un tableau X qui contient les caractéristiques des maisons, et un tableau y avec la variable cible, dans ce cas, la colonne `'Prix'`.

> Nous exclurons en tout cas la colonne `'Address'` car elle ne contient que des informations textuelles que le modèle de régression linéaire ne peut pas utiliser.

> **QUESTION**: Quelle autre colonne exclueriez-vous car peu informative?

## Tableaux X et y

In [ ]:
# Garder toutes les colonnes sauf 'Address' et 'Price'
X = USAhousing.drop(['Address','Price'],axis='columns')
# Garder seulement la colonne 'Price'
y = USAhousing['Price']

## 🧱 Division en ensembles d'entraînement et de test

Maintenant, divisons les données en un ensemble d'entraînement et un ensemble de test. Nous allons entraîner notre modèle sur l'ensemble d'entraînement, puis utiliser l'ensemble de test pour évaluer le modèle.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 📦 Préparation des données pour la régression linéaire

> La régression linéaire a été étudiée en profondeur, et il existe beaucoup de littérature sur la manière dont vos données doivent être structurées pour tirer le meilleur parti du modèle.

- **Hypothèse de linéarité.** La régression linéaire suppose que la relation entre vos variables d'entrée et de sortie est linéaire. Elle ne prend pas en charge autre chose. Cela peut sembler évident, mais il est bon de s'en souvenir lorsque vous avez de nombreuses variables. Vous devrez peut-être transformer les données pour rendre la relation linéaire (par exemple, une transformation logarithmique pour une relation exponentielle).

- **Éliminer le bruit.** La régression linéaire suppose que vos variables d'entrée et de sortie ne sont pas bruitées. Envisagez d'utiliser des opérations de nettoyage des données qui vous permettent de mieux exposer et clarifier le signal dans vos données. C'est particulièrement important pour la variable de sortie, et vous voudrez éliminer les valeurs aberrantes (outliers) dans la variable de sortie (y) si possible.

- **Éliminer la collinéarité.** La régression linéaire surajustera vos données lorsque vos variables d'entrée sont fortement corrélées. Envisagez de calculer les corrélations par paires de vos données d'entrée et de supprimer les corrélations les plus élevées.

- **Distributions gaussiennes.** La régression linéaire fera des prédictions plus fiables si vos variables d'entrée et de sortie ont une distribution gaussienne. Vous pouvez obtenir des avantages en utilisant des transformations (par exemple, fonction logarithmique) sur vos variables pour rendre leur distribution plus proche de celle d'une gaussienne.

- **Redimensionner les entrées:** La régression linéaire fera souvent des prédictions plus fiables si vous redimensionnez les variables d'entrée en utilisant la standardisation ou la normalisation.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Standardisation
pipeline = Pipeline([
    ('std_scalar', StandardScaler())
])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

# ✔️ Regression Linéaire

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

## ✔️ Évaluation du modèle

Évaluons le modèle en examinant ses coefficients et en comprenant comment les interpréter.

In [ ]:
# Afficher l'intercept
print(lin_reg.intercept_)

In [ ]:
# Afficher les coefficient de le régression
coeff_df = pd.DataFrame(lin_reg.coef_, X.columns, columns=['Coefficient'])
coeff_df

**Question**: Comment interprétez-vous ces coefficients? 
- En maintenant constantes toutes les autres caractéristiques, une augmentation de 1 unité du **Revenu moyen de la zone** est associée à une **augmentation de X1 $**.
- En maintenant constantes toutes les autres caractéristiques, une augmentation de 1 unité de **l'Âge moyen des maisons de la zone** est associée à une **augmentation de X2 $**.
- En maintenant constantes toutes les autres caractéristiques, une augmentation de 1 unité du **Nombre moyen de pièces dans la zone** est associée à une **augmentation de X3 $**.
- En maintenant constantes toutes les autres caractéristiques, une augmentation de 1 unité du **Nombre moyen de chambres dans la zone** est associée à une **augmentation de X4 $**.
- En maintenant constantes toutes les autres caractéristiques, une augmentation de 1 unité de **la Population de la zone** est associée à une **augmentation de X5 $**.

<details>
<summary>Aide</summary>
N'oubliez pas la standardisation faite au début sur les données!

Les écart types utilisés pour la standardisation: `pipeline.named_steps['std_scalar'].scale_`

</details>

In [ ]:
# Déviations Standards (Ecarts Types) des caractéristiques (utilisées lors de la normalisation)
# CODE ICI

# Une augmentation de 1 unité de la feature est associée à : ...
# CODE ICI


## ✔️ Prédictions de notre modèle

Obtenons des prédictions à partir de notre ensemble de test et voyons à quel point elles sont précises !

In [ ]:
pred = lin_reg.predict(X_test)

In [ ]:
pd.DataFrame({'True Values': y_test, 'Predicted Values': pred}).hvplot.scatter(x='True Values', y='Predicted Values')

**Residual Histogram**

In [ ]:
pd.DataFrame({'Error Values': (y_test - pred)}).hvplot.kde()

## ✔️ Métriques d'évaluation de la régression

Voici trois métriques d'évaluation courantes pour les problèmes de régression :

> - **Erreur absolue moyenne** (MAE) est la moyenne de la valeur absolue des erreurs :
$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$

> - **Erreur quadratique moyenne** (MSE) est la moyenne des erreurs au carré :
$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

> - **Erreur quadratique moyenne racine** (RMSE) est la racine carrée de la moyenne des erreurs au carré :
$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

> 📌 Comparaison de ces métriques :
- **MAE** est la plus facile à comprendre, car il s'agit de l'erreur moyenne.
- **MSE** est plus populaire que MAE, car MSE "punit" les erreurs plus importantes, ce qui est souvent utile dans le monde réel.
- **RMSE** est encore plus populaire que MSE, car RMSE est interprétable dans les unités "y".

> Toutes ces métriques sont des **fonctions de coût**, que nous souhaitons minimiser.

> Nous utiliserons également la métrique du [R2 score](https://kobia.fr/regression-metrics-r2-score/#:~:text=Le%20R2%20score%2C%20aussi%20appel%C3%A9,MSE%20(Mean%20Squared%20Error).). Cette métrique est une version “normalisée” de la MSE (Mean Squared Error). On peut voir le R2 comme l’erreur du modèle divisé par l’erreur d’un modèle basique qui prédit tout le temps la moyenne de la variable à prédire:
$$1 - \frac{\sum_{i=1}^n (y_i - \hat{y}_i)^2}{\sum_{i=1}^n (y_i - \bar{y})^2}$$

> Le score R2 est d’autant plus élevé que le modèle est performant, et vaut au maximum 100%, lorsque toutes les prédictions sont exactes. Un modèle simple prédisant tout le temps la valeur moyenne atteint un score R2 de 0%.

In [ ]:
from sklearn import metrics

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
test_pred = lin_reg.predict(X_test)
train_pred = lin_reg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred)]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square'])

# ✔️ Random Forest Regressor
Est-ce qu'un modèle non-linéaire plus complexe aurait fait mieux?

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=1000)
rf_reg.fit(X_train, y_train)

test_pred = rf_reg.predict(X_test)
train_pred = rf_reg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Random Forest Regressor", *evaluate(y_test, test_pred)]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square'])
results_df = pd.concat([results_df, results_df_2], ignore_index=True)


In [ ]:
results_df

**Question**: Comment interprétez-vous ces resultats? 

- Quel modèle a les meilleures performances sur les données de test?

- Qu'en est-il des performances sur les données d'entrainement?

- Discutez le biais et la variance des deux modèles et le risque de sur-apprentissage.

<details>
<summary>Rappel</summary>

- Un modèle **surajusté** (overfitted) correspond trop étroitement aux données d'entraînement, et ne généralise pas efficacement.

- **Biais** : Erreur due à des hypothèses simplificatrices du modèle, conduisant à des prédictions erronées sur les données d'entraînement et de test.

- **Variance** : Sensibilité excessive aux variations dans les données d'entraînement, entraînant des 
fluctuations importantes des prédictions entre différents ensembles de données.
</details>

# 📝 Résumé
Dans ce notebook, vous avez utiliser deux algorithmes de régression pour l'apprentissage automatique.

Vous avez couvert les aspects suivants:
> - Le modèle courant de régression linéaire
> - L'exploration et la préparation des données avant leur utilisation pour la régression linéaire.
> - L'entrainement du modèle de régression linéaire
> - L'interprétation du modèle
> - L'évaluation du modèle 
> - La comparaison avec les prédictions d'un modèle plus complexe


# 🔗 References:
- [Scikit-learn library](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)
- [Linear Regression for Machine Learning by Jason Brownlee PhD](https://machinelearningmastery.com/linear-regression-for-machine-learning/)
- [Kaggle - Practical Introduction to 10 Regression Algorithm](https://www.kaggle.com/code/faressayah/practical-introduction-to-10-regression-algorithm)